# Create Folders

In [ ]:
!mkdir log
!mkdir model
!mkdir result

# Import

In [ ]:
import gc
import os
import time
import torch
import datetime
from utils import *
import torch.nn as nn
from Model import Unet
from Dataset import NavigationDataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from Loss import TverskyLoss, WeightedCrossEntropyLoss

# Global variable

In [ ]:
IMG_WIDTH = 428
IMG_HIGHT = 240

# Inference

In [ ]:
def inference(model,public_dataloader,  private_dataloader):
    if not os.path.isdir("./result"):
        os.mkdir("result")
    floder_path = "./result/"
    threshold = 176
    toimg = transforms.ToPILImage()
    resize = transforms.Resize((IMG_HIGHT, IMG_WIDTH), antialias=True)

    model.eval()
    with torch.no_grad():
        # public
        for i, data in enumerate(public_dataloader):
            images = data[0]
            images = images.to(get_device())

            outputs = model(images)[0][0]
            cv2_img = convert_from_image_to_cv2(toimg(resize(outputs)))

            _, cv2_img = cv2.threshold(cv2_img, threshold, 255, cv2.THRESH_BINARY)
            if i <360:
                cv2.imwrite(floder_path + f"PUB_RI_{2000000 + i}.png", cv2_img)
            else:
                cv2.imwrite(floder_path + f"PUB_RO_{2000000 + i}.png", cv2_img)
            processbar(i+1, 720, 30, f"{i+1:4d}/{720:4d} predicted.")

        # private
        for i, data in enumerate(private_dataloader):
            images = data[0]
            images = images.to(get_device())

            outputs = model(images)[0][0]
            cv2_img = convert_from_image_to_cv2(toimg(resize(outputs)))

            _, cv2_img = cv2.threshold(cv2_img, threshold, 255, cv2.THRESH_BINARY)
            if i <360:
                cv2.imwrite(floder_path + f"PRI_RI_{2000000 + i}.png", cv2_img)
            else:
                cv2.imwrite(floder_path + f"PRI_RO_{2000000 + i}.png", cv2_img)
            
            processbar(i+1, 720, 30, f"{i+1:4d}/{720:4d} predicted.")

# Main

In [ ]:
def main():
    # hyper parameter
    batch_size = 1

    # dataset
    print("Loading training dataset...")
    pub_test_set = NavigationDataset(mode="TEST", dataset_path="./Dataset/",test_mode="PUB")
    pri_test_set = NavigationDataset(mode="TEST", dataset_path="./PrivateDataset/",test_mode="PRI")
    
    pub_test_dataloader = DataLoader(pub_test_set, batch_size= batch_size, shuffle= False)
    pri_test_dataloader = DataLoader(pri_test_set, batch_size= batch_size, shuffle= False)
    print("Loading complete.")

    # model
    model = Unet(3, 1)
    model.load_state_dict({k.replace('module.',''):v for k,v in torch.load("./model/best_model.pth").items()})
    model.to(get_device())
    model = nn.DataParallel(model, device_ids=[i for i in range(torch.cuda.device_count())])

    # inference
    print("Start inferencing...")
    start_time = datetime.datetime.now()
    inference(model, public_dataloader=pub_test_dataloader, private_dataloader=pri_test_dataloader)
    end_time = datetime.datetime.now()
    print(f"Inferencing complete.\nCost {str(end_time - start_time)} .")

if __name__ == "__main__":
    main()

# Package result

In [ ]:
!zip -r result.zip result